<a href="https://colab.research.google.com/github/sher1w/Regional-Fake-News-Models/blob/main/airavata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers accelerate torch sentencepiece pandas scikit-learn


In [ ]:
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get("ColabToken")  # your saved secret
login(token=hf_token)


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "ai4bharat/Airavata"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/968k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.85M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.81G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
from google.colab import files

uploaded = files.upload()  # upload dataset.xlsx


Saving final_cleaned_data.csv to final_cleaned_data (1).csv


In [ ]:
import pandas as pd

df = pd.read_csv("cleaned_file2 (1).csv")

df = df.dropna()
df.head()


,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile
3,4,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते ...,"defamation,offensive"
4,5,- अनलॉक-4 के लिए गाइडलाइन्स जारी - 7 सितंबर से...,non-hostile


In [ ]:
def fake_news_prompt(text):
    return f"""
You are a professional fact-checking system.

Classify the following Hindi news as either:
FAKE or REAL

Respond with ONLY one word: FAKE or REAL.

News:
{text}

Answer:
"""


In [ ]:
def predict_fake_news(text):
    prompt = fake_news_prompt(text)

    output = pipe(
        prompt,
        max_new_tokens=5,
        do_sample=False,
        truncation=True
    )[0]["generated_text"]

    answer = output.split("Answer:")[-1].strip().split()[0]
    return answer.upper()


In [ ]:
df = df.rename(columns={
    "Post": "text",
    "Labels Set": "label"
})

print(df.columns)


Index(['Text', 'Label', 'Word_Count'], dtype='object')


In [ ]:
# Run this right after loading your CSV/dataframe
df.columns = df.columns.str.lower()

# Now "text" will work perfectly!
df_small = df.sample(100, random_state=42)
df_small["prediction"] = df_small["text"].apply(predict_fake_news)

In [ ]:
# 1. Standardize column names to lowercase
df.columns = df.columns.str.lower()

# 2. Sample the data
df_small = df.sample(100, random_state=42)

# 3. Use lowercase "text" (since we just renamed it in step 1)
df_small["prediction"] = df_small["text"].apply(predict_fake_news)

# 4. Map the results (using the mapping we discussed)
mapping = {"FAKE": 0, "REAL": 1}
df_small["prediction"] = df_small["prediction"].str.upper().map(mapping)

# 5. Check the result
print(df_small[["text", "label", "prediction"]].head())

                                                    text label  prediction
9115   रियो डी जनेरियो - सिमोन मैनुअल ने गुरुवार रात ...     0           1
12443  आज तक न्यूज़ चैनल एक नंबर से 10 नंबर पर पहुंच ...     0           0
16721  एनमॉर्गन प्रोजेक्ट वेरिटास जेम्स ओकीफ के दिमाग...     1           0
13342  भारत ने कहा है कि सुरक्षा परिषद में सुधार की अ...     0           1
9386   राम मंदिर निर्माण शुरू होने के वक्त एक मस्जिद ...     0           1


In [ ]:
# Convert y_true from Strings to Integers
y_true = df_small[LABEL_COLUMN].astype(int)
y_pred = df_small["prediction"]

# Now this will work!
print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred))

Accuracy: 0.5

Classification Report:

              precision    recall  f1-score   support

           0       0.56      0.42      0.48        55
           1       0.46      0.60      0.52        45

    accuracy                           0.50       100
   macro avg       0.51      0.51      0.50       100
weighted avg       0.51      0.50      0.50       100



In [ ]:
df_small.to_excel("airavata_fake_news_results.xlsx", index=False)
print("Saved results to airavata_fake_news_results.xlsx")


Saved results to airavata_fake_news_results.xlsx


In [ ]:
!pip install -q transformers accelerate torch sentencepiece pandas scikit-learn
from huggingface_hub import login
from google.colab import userdata
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sklearn.metrics import accuracy_score, classification_report

# Login
hf_token = userdata.get("ColabToken")
login(token=hf_token)

# Load Airavata
model_id = "ai4bharat/Airavata"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def fake_news_prompt(text):
    # We provide examples (Few-Shot) to help the model understand Hindi context
    return f"""निर्देश: नीचे दिए गए समाचार को 'FAKE' या 'REAL' के रूप में वर्गीकृत करें। केवल एक शब्द में उत्तर दें।

उदाहरण 1:
समाचार: दिल्ली में कल से पेट्रोल 10 रुपये सस्ता होगा।
उत्तर: FAKE

उदाहरण 2:
समाचार: भारत ने जी20 शिखर सम्मेलन की मेजबानी की।
उत्तर: REAL

समाचार: {text}
उत्तर:"""

def predict_fake_news(text):
    prompt = fake_news_prompt(text)

    # We use a small max_new_tokens because we only want one word
    output = pipe(
        prompt,
        max_new_tokens=10,
        do_sample=False,
        truncation=True
    )[0]["generated_text"]

    # Extract the part after the last "उत्तर:"
    raw_answer = output.split("उत्तर:")[-1].strip().upper()

    # Robust check: if the word FAKE is anywhere in the response, count it
    if "FAKE" in raw_answer:
        return "FAKE"
    elif "REAL" in raw_answer:
        return "REAL"
    else:
        return "UNKNOWN" # To catch errors

In [ ]:
import pandas as pd

# Load your data (assuming df is already loaded from your Excel/CSV)
# Standardize names
df.columns = df.columns.str.lower()
if 'post' in df.columns:
    df = df.rename(columns={"post": "text", "labels set": "label"})

# Take a sample
df_small = df.sample(100, random_state=42).copy()

print("Processing 100 samples... this may take a few minutes.")
df_small["prediction_raw"] = df_small["text"].apply(predict_fake_news)

# Map predictions to match the integer labels (0 and 1)
mapping = {"FAKE": 0, "REAL": 1}
df_small["prediction"] = df_small["prediction_raw"].map(mapping)

# Ensure y_true is integer
y_true = df_small["label"].astype(int)
y_pred = df_small["prediction"].fillna(0).astype(int) # Fill unknown as 0 (Fake)

# Final Results
print("\n" + "="*30)
print(f"FINAL ACCURACY: {accuracy_score(y_true, y_pred):.2%}")
print("="*30)
print("\nFull Report:\n", classification_report(y_true, y_pred))